# Constant

# Load data

In [51]:
import pandas as pd

data = pd.read_csv("dataset2020.csv")
mapping = {'pe-legit': 0, 'pe-malicious': 1}
data.iloc[:,0].replace(mapping, inplace=True)

X=data.iloc[:,1:].values
Y=data.iloc[:,0].values


# process data

In [52]:
from numpy import array
import numpy as np
from tensorflow.keras.utils import *

feature=array(X)
label=array(Y)
label = np.reshape(label, (label.shape[0], 1))
label = to_categorical(label, num_classes = 2)

arr = []
for i in feature:
  arr.append(np.reshape(i, (1, 486)).tolist())
feature = np.array(arr)

print(label.shape, feature.shape)
print(label)


(49128, 2) (49128, 1, 486)
[[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [1. 0.]]


# Split train-test data

In [53]:
from sklearn.model_selection import train_test_split
X_train ,X_test, Y_train,Y_test = train_test_split(feature, label,test_size=1 / 5, random_state=0)
print(Y_test)

[[1. 0.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


# Create model

In [54]:
import keras.backend as K
def f1_score(y_true, y_pred, threshold_shift=0):
    beta = 2

    y_pred = K.clip(y_pred, 0, 1)

    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin), axis=1) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)), axis=1)
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)), axis=1)

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return K.mean((beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon()))

# Activation Tanh 32

In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from keras.metrics import *
import keras_metrics as km

model = Sequential()
model.add(LSTM(32, input_shape=(X_train.shape[1:]), return_sequences=False, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile( loss="binary_crossentropy",
              optimizer=RMSprop(lr=0.001), 
              metrics=['accuracy'])

history = model.fit(x=X_train, y=Y_train,batch_size=36,
                    epochs=50, validation_split = 0.2, shuffle=True)

from sklearn.metrics import *
y_pred = model.predict(X_test)

label_pred = np.argmax(y_pred, axis = -1)
label_true = np.argmax(Y_test, axis = -1)
matrix = confusion_matrix(label_true, label_pred)
print(matrix)
print("acc : ", accuracy_score(label_true, label_pred))
print("f1 : ", f1_score(label_true, label_pred, average='macro'))
print("recall : ", recall_score(label_true, label_pred, average='macro'))
print("precision : ", precision_score(label_true, label_pred, average='macro'))

import matplotlib.pyplot as plt
from matplotlib.colors import *
 
def plot_confusion_matrix(cm, classes,
                        normalize=True,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
target_names =["Legit","Malicious"]
plot_confusion_matrix(cm=matrix, classes=target_names, normalize=False)

from matplotlib import pyplot
 
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()
 
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()

tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fn


# Activation Tanh 64

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from keras.metrics import *
import keras_metrics as km

model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1:]), return_sequences=False, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile( loss="binary_crossentropy",
              optimizer=RMSprop(lr=0.001), 
              metrics=['accuracy'])

history = model.fit(x=X_train, y=Y_train,batch_size=36,
                    epochs=50, validation_split = 0.2, shuffle=True)

from sklearn.metrics import *
y_pred = model.predict(X_test)

label_pred = np.argmax(y_pred, axis = -1)
label_true = np.argmax(Y_test, axis = -1)
matrix = confusion_matrix(label_true, label_pred)
print(matrix)
print("acc : ", accuracy_score(label_true, label_pred))
print("f1 : ", f1_score(label_true, label_pred, average='macro'))
print("recall : ", recall_score(label_true, label_pred, average='macro'))
print("precision : ", precision_score(label_true, label_pred, average='macro'))

import matplotlib.pyplot as plt
from matplotlib.colors import *
 
def plot_confusion_matrix(cm, classes,
                        normalize=True,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
target_names =["Legit","Malicious"]
plot_confusion_matrix(cm=matrix, classes=target_names, normalize=False)

from matplotlib import pyplot
 
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()
 
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()

# Activation tanh 128

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from keras.metrics import *
import keras_metrics as km

model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1:]), return_sequences=False, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile( loss="binary_crossentropy",
              optimizer=RMSprop(lr=0.001), 
              metrics=['accuracy'])

history = model.fit(x=X_train, y=Y_train,batch_size=36,
                    epochs=50, validation_split = 0.2, shuffle=True)

from sklearn.metrics import *
y_pred = model.predict(X_test)

label_pred = np.argmax(y_pred, axis = -1)
label_true = np.argmax(Y_test, axis = -1)
matrix = confusion_matrix(label_true, label_pred)
print(matrix)
print("acc : ", accuracy_score(label_true, label_pred))
print("f1 : ", f1_score(label_true, label_pred, average='macro'))
print("recall : ", recall_score(label_true, label_pred, average='macro'))
print("precision : ", precision_score(label_true, label_pred, average='macro'))

import matplotlib.pyplot as plt
from matplotlib.colors import *
 
def plot_confusion_matrix(cm, classes,
                        normalize=True,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
target_names =["Legit","Malicious"]
plot_confusion_matrix(cm=matrix, classes=target_names, normalize=False)

from matplotlib import pyplot
 
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()
 
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()

# Acitvation tanh 512

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from keras.metrics import *
import keras_metrics as km

model = Sequential()
model.add(LSTM(512, input_shape=(X_train.shape[1:]), return_sequences=False, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile( loss="binary_crossentropy",
              optimizer=RMSprop(lr=0.001), 
              metrics=['accuracy'])

history = model.fit(x=X_train, y=Y_train,batch_size=36,
                    epochs=50, validation_split = 0.2, shuffle=True)

from sklearn.metrics import *
y_pred = model.predict(X_test)

label_pred = np.argmax(y_pred, axis = -1)
label_true = np.argmax(Y_test, axis = -1)
matrix = confusion_matrix(label_true, label_pred)
print(matrix)
print("acc : ", accuracy_score(label_true, label_pred))
print("f1 : ", f1_score(label_true, label_pred, average='macro'))
print("recall : ", recall_score(label_true, label_pred, average='macro'))
print("precision : ", precision_score(label_true, label_pred, average='macro'))

import matplotlib.pyplot as plt
from matplotlib.colors import *
 
def plot_confusion_matrix(cm, classes,
                        normalize=True,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
target_names =["Legit","Malicious"]
plot_confusion_matrix(cm=matrix, classes=target_names, normalize=False)

from matplotlib import pyplot
 
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()
 
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()

# Relu 32

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from keras.metrics import *
import keras_metrics as km

model = Sequential()
model.add(LSTM(32, input_shape=(X_train.shape[1:]), return_sequences=False, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile( loss="binary_crossentropy",
              optimizer=RMSprop(lr=0.001), 
              metrics=['accuracy'])

history = model.fit(x=X_train, y=Y_train,batch_size=36,
                    epochs=50, validation_split = 0.2, shuffle=True)

from sklearn.metrics import *
y_pred = model.predict(X_test)

label_pred = np.argmax(y_pred, axis = -1)
label_true = np.argmax(Y_test, axis = -1)
matrix = confusion_matrix(label_true, label_pred)
print(matrix)
print("acc : ", accuracy_score(label_true, label_pred))
print("f1 : ", f1_score(label_true, label_pred, average='macro'))
print("recall : ", recall_score(label_true, label_pred, average='macro'))
print("precision : ", precision_score(label_true, label_pred, average='macro'))

import matplotlib.pyplot as plt
from matplotlib.colors import *
 
def plot_confusion_matrix(cm, classes,
                        normalize=True,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
target_names =["Legit","Malicious"]
plot_confusion_matrix(cm=matrix, classes=target_names, normalize=False)

from matplotlib import pyplot
 
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()
 
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()

# Relu 64

In [58]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from keras.metrics import *
import keras_metrics as km

model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1:]), return_sequences=False, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile( loss="binary_crossentropy",
              optimizer=RMSprop(lr=0.001), 
              metrics=['accuracy'])

history = model.fit(x=X_train, y=Y_train,batch_size=36,
                    epochs=50, validation_split = 0.2, shuffle=True)

from sklearn.metrics import *
y_pred = model.predict(X_test)

label_pred = np.argmax(y_pred, axis = -1)
label_true = np.argmax(Y_test, axis = -1)
matrix = confusion_matrix(label_true, label_pred)
print(matrix)
print("acc : ", accuracy_score(label_true, label_pred))
print("f1 : ", f1_score(label_true, label_pred, average='macro'))
print("recall : ", recall_score(label_true, label_pred, average='macro'))
print("precision : ", precision_score(label_true, label_pred, average='macro'))

import matplotlib.pyplot as plt
from matplotlib.colors import *
 
def plot_confusion_matrix(cm, classes,
                        normalize=True,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
target_names =["Legit","Malicious"]
plot_confusion_matrix(cm=matrix, classes=target_names, normalize=False)

from matplotlib import pyplot
 
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()
 
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()

Epoch 1/50
662/874 [=====================>........] - ETA: 0s - loss: 0.4104 - accuracy: 0.8138

KeyboardInterrupt: 

# Relu 128

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from keras.metrics import *
import keras_metrics as km

model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1:]), return_sequences=False, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile( loss="binary_crossentropy",
              optimizer=RMSprop(lr=0.001), 
              metrics=['accuracy'])

history = model.fit(x=X_train, y=Y_train,batch_size=36,
                    epochs=50, validation_split = 0.2, shuffle=True)

from sklearn.metrics import *
y_pred = model.predict(X_test)

label_pred = np.argmax(y_pred, axis = -1)
label_true = np.argmax(Y_test, axis = -1)
matrix = confusion_matrix(label_true, label_pred)
print(matrix)
print("acc : ", accuracy_score(label_true, label_pred))
print("f1 : ", f1_score(label_true, label_pred, average='macro'))
print("recall : ", recall_score(label_true, label_pred, average='macro'))
print("precision : ", precision_score(label_true, label_pred, average='macro'))

import matplotlib.pyplot as plt
from matplotlib.colors import *
 
def plot_confusion_matrix(cm, classes,
                        normalize=True,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
target_names =["Legit","Malicious"]
plot_confusion_matrix(cm=matrix, classes=target_names, normalize=False)

from matplotlib import pyplot
 
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()
 
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()

# Relu 512

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from keras.metrics import *
import keras_metrics as km

model = Sequential()
model.add(LSTM(512, input_shape=(X_train.shape[1:]), return_sequences=False, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile( loss="binary_crossentropy",
              optimizer=RMSprop(lr=0.001), 
              metrics=['accuracy'])

history = model.fit(x=X_train, y=Y_train,batch_size=36,
                    epochs=50, validation_split = 0.2, shuffle=True)

from sklearn.metrics import *
y_pred = model.predict(X_test)

label_pred = np.argmax(y_pred, axis = -1)
label_true = np.argmax(Y_test, axis = -1)
matrix = confusion_matrix(label_true, label_pred)
print(matrix)
print("acc : ", accuracy_score(label_true, label_pred))
print("f1 : ", f1_score(label_true, label_pred, average='macro'))
print("recall : ", recall_score(label_true, label_pred, average='macro'))
print("precision : ", precision_score(label_true, label_pred, average='macro'))

import matplotlib.pyplot as plt
from matplotlib.colors import *
 
def plot_confusion_matrix(cm, classes,
                        normalize=True,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
target_names =["Legit","Malicious"]
plot_confusion_matrix(cm=matrix, classes=target_names, normalize=False)

from matplotlib import pyplot
 
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()
 
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()

# elu 32

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from keras.metrics import *
import keras_metrics as km

model = Sequential()
model.add(LSTM(32, input_shape=(X_train.shape[1:]), return_sequences=False, activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile( loss="binary_crossentropy",
              optimizer=RMSprop(lr=0.001), 
              metrics=['accuracy'])

history = model.fit(x=X_train, y=Y_train,batch_size=36,
                    epochs=50, validation_split = 0.2, shuffle=True)

from sklearn.metrics import *
y_pred = model.predict(X_test)

label_pred = np.argmax(y_pred, axis = -1)
label_true = np.argmax(Y_test, axis = -1)
matrix = confusion_matrix(label_true, label_pred)
print(matrix)
print("acc : ", accuracy_score(label_true, label_pred))
print("f1 : ", f1_score(label_true, label_pred, average='macro'))
print("recall : ", recall_score(label_true, label_pred, average='macro'))
print("precision : ", precision_score(label_true, label_pred, average='macro'))

import matplotlib.pyplot as plt
from matplotlib.colors import *
 
def plot_confusion_matrix(cm, classes,
                        normalize=True,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
target_names =["Legit","Malicious"]
plot_confusion_matrix(cm=matrix, classes=target_names, normalize=False)

from matplotlib import pyplot
 
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()
 
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()

# elu 64 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from keras.metrics import *
import keras_metrics as km

model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1:]), return_sequences=False, activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile( loss="binary_crossentropy",
              optimizer=RMSprop(lr=0.001), 
              metrics=['accuracy'])

history = model.fit(x=X_train, y=Y_train,batch_size=36,
                    epochs=50, validation_split = 0.2, shuffle=True)

from sklearn.metrics import *
y_pred = model.predict(X_test)

label_pred = np.argmax(y_pred, axis = -1)
label_true = np.argmax(Y_test, axis = -1)
matrix = confusion_matrix(label_true, label_pred)
print(matrix)
print("acc : ", accuracy_score(label_true, label_pred))
print("f1 : ", f1_score(label_true, label_pred, average='macro'))
print("recall : ", recall_score(label_true, label_pred, average='macro'))
print("precision : ", precision_score(label_true, label_pred, average='macro'))

import matplotlib.pyplot as plt
from matplotlib.colors import *
 
def plot_confusion_matrix(cm, classes,
                        normalize=True,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
target_names =["Legit","Malicious"]
plot_confusion_matrix(cm=matrix, classes=target_names, normalize=False)

from matplotlib import pyplot
 
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()
 
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()

# elu 128

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from keras.metrics import *
import keras_metrics as km

model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1:]), return_sequences=False, activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile( loss="binary_crossentropy",
              optimizer=RMSprop(lr=0.001), 
              metrics=['accuracy'])

history = model.fit(x=X_train, y=Y_train,batch_size=36,
                    epochs=50, validation_split = 0.2, shuffle=True)

from sklearn.metrics import *
y_pred = model.predict(X_test)

label_pred = np.argmax(y_pred, axis = -1)
label_true = np.argmax(Y_test, axis = -1)
matrix = confusion_matrix(label_true, label_pred)
print(matrix)
print("acc : ", accuracy_score(label_true, label_pred))
print("f1 : ", f1_score(label_true, label_pred, average='macro'))
print("recall : ", recall_score(label_true, label_pred, average='macro'))
print("precision : ", precision_score(label_true, label_pred, average='macro'))

import matplotlib.pyplot as plt
from matplotlib.colors import *
 
def plot_confusion_matrix(cm, classes,
                        normalize=True,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
target_names =["Legit","Malicious"]
plot_confusion_matrix(cm=matrix, classes=target_names, normalize=False)

from matplotlib import pyplot
 
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()
 
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()

# elu 512

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from keras.metrics import *
import keras_metrics as km

model = Sequential()
model.add(LSTM(512, input_shape=(X_train.shape[1:]), return_sequences=False, activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile( loss="binary_crossentropy",
              optimizer=RMSprop(lr=0.001), 
              metrics=['accuracy'])

history = model.fit(x=X_train, y=Y_train,batch_size=36,
                    epochs=50, validation_split = 0.2, shuffle=True)

from sklearn.metrics import *
y_pred = model.predict(X_test)

label_pred = np.argmax(y_pred, axis = -1)
label_true = np.argmax(Y_test, axis = -1)
matrix = confusion_matrix(label_true, label_pred)
print(matrix)
print("acc : ", accuracy_score(label_true, label_pred))
print("f1 : ", f1_score(label_true, label_pred, average='macro'))
print("recall : ", recall_score(label_true, label_pred, average='macro'))
print("precision : ", precision_score(label_true, label_pred, average='macro'))

import matplotlib.pyplot as plt
from matplotlib.colors import *
 
def plot_confusion_matrix(cm, classes,
                        normalize=True,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
target_names =["Legit","Malicious"]
plot_confusion_matrix(cm=matrix, classes=target_names, normalize=False)

from matplotlib import pyplot
 
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()
 
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()

# selu 32

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from keras.metrics import *
import keras_metrics as km

model = Sequential()
model.add(LSTM(32, input_shape=(X_train.shape[1:]), return_sequences=False, activation='selu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile( loss="binary_crossentropy",
              optimizer=RMSprop(lr=0.001), 
              metrics=['accuracy'])

history = model.fit(x=X_train, y=Y_train,batch_size=36,
                    epochs=50, validation_split = 0.2, shuffle=True)

from sklearn.metrics import *
y_pred = model.predict(X_test)

label_pred = np.argmax(y_pred, axis = -1)
label_true = np.argmax(Y_test, axis = -1)
matrix = confusion_matrix(label_true, label_pred)
print(matrix)
print("acc : ", accuracy_score(label_true, label_pred))
print("f1 : ", f1_score(label_true, label_pred, average='macro'))
print("recall : ", recall_score(label_true, label_pred, average='macro'))
print("precision : ", precision_score(label_true, label_pred, average='macro'))

import matplotlib.pyplot as plt
from matplotlib.colors import *
 
def plot_confusion_matrix(cm, classes,
                        normalize=True,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
target_names =["Legit","Malicious"]
plot_confusion_matrix(cm=matrix, classes=target_names, normalize=False)

from matplotlib import pyplot
 
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()
 
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()

# selu 64

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from keras.metrics import *
import keras_metrics as km

model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1:]), return_sequences=False, activation='selu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile( loss="binary_crossentropy",
              optimizer=RMSprop(lr=0.001), 
              metrics=['accuracy'])

history = model.fit(x=X_train, y=Y_train,batch_size=36,
                    epochs=50, validation_split = 0.2, shuffle=True)

from sklearn.metrics import *
y_pred = model.predict(X_test)

label_pred = np.argmax(y_pred, axis = -1)
label_true = np.argmax(Y_test, axis = -1)
matrix = confusion_matrix(label_true, label_pred)
print(matrix)
print("acc : ", accuracy_score(label_true, label_pred))
print("f1 : ", f1_score(label_true, label_pred, average='macro'))
print("recall : ", recall_score(label_true, label_pred, average='macro'))
print("precision : ", precision_score(label_true, label_pred, average='macro'))

import matplotlib.pyplot as plt
from matplotlib.colors import *
 
def plot_confusion_matrix(cm, classes,
                        normalize=True,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
target_names =["Legit","Malicious"]
plot_confusion_matrix(cm=matrix, classes=target_names, normalize=False)

from matplotlib import pyplot
 
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()
 
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()

# selu 128

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from keras.metrics import *
import keras_metrics as km

model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1:]), return_sequences=False, activation='selu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile( loss="binary_crossentropy",
              optimizer=RMSprop(lr=0.001), 
              metrics=['accuracy'])

history = model.fit(x=X_train, y=Y_train,batch_size=36,
                    epochs=50, validation_split = 0.2, shuffle=True)

from sklearn.metrics import *
y_pred = model.predict(X_test)

label_pred = np.argmax(y_pred, axis = -1)
label_true = np.argmax(Y_test, axis = -1)
matrix = confusion_matrix(label_true, label_pred)
print(matrix)
print("acc : ", accuracy_score(label_true, label_pred))
print("f1 : ", f1_score(label_true, label_pred, average='macro'))
print("recall : ", recall_score(label_true, label_pred, average='macro'))
print("precision : ", precision_score(label_true, label_pred, average='macro'))

import matplotlib.pyplot as plt
from matplotlib.colors import *
 
def plot_confusion_matrix(cm, classes,
                        normalize=True,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
target_names =["Legit","Malicious"]
plot_confusion_matrix(cm=matrix, classes=target_names, normalize=False)

from matplotlib import pyplot
 
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()
 
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()

# selu 512

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from keras.metrics import *
import keras_metrics as km

model = Sequential()
model.add(LSTM(512, input_shape=(X_train.shape[1:]), return_sequences=False, activation='selu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile( loss="binary_crossentropy",
              optimizer=RMSprop(lr=0.001), 
              metrics=['accuracy'])

history = model.fit(x=X_train, y=Y_train,batch_size=36,
                    epochs=50, validation_split = 0.2, shuffle=True)

from sklearn.metrics import *
y_pred = model.predict(X_test)

label_pred = np.argmax(y_pred, axis = -1)
label_true = np.argmax(Y_test, axis = -1)
matrix = confusion_matrix(label_true, label_pred)
print(matrix)
print("acc : ", accuracy_score(label_true, label_pred))
print("f1 : ", f1_score(label_true, label_pred, average='macro'))
print("recall : ", recall_score(label_true, label_pred, average='macro'))
print("precision : ", precision_score(label_true, label_pred, average='macro'))

import matplotlib.pyplot as plt
from matplotlib.colors import *
 
def plot_confusion_matrix(cm, classes,
                        normalize=True,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
target_names =["Legit","Malicious"]
plot_confusion_matrix(cm=matrix, classes=target_names, normalize=False)

from matplotlib import pyplot
 
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()
 
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()

# softsign 32

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from keras.metrics import *
import keras_metrics as km

model = Sequential()
model.add(LSTM(32, input_shape=(X_train.shape[1:]), return_sequences=False, activation='softsign'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile( loss="binary_crossentropy",
              optimizer=RMSprop(lr=0.001), 
              metrics=['accuracy'])

history = model.fit(x=X_train, y=Y_train,batch_size=36,
                    epochs=50, validation_split = 0.2, shuffle=True)

from sklearn.metrics import *
y_pred = model.predict(X_test)

label_pred = np.argmax(y_pred, axis = -1)
label_true = np.argmax(Y_test, axis = -1)
matrix = confusion_matrix(label_true, label_pred)
print(matrix)
print("acc : ", accuracy_score(label_true, label_pred))
print("f1 : ", f1_score(label_true, label_pred, average='macro'))
print("recall : ", recall_score(label_true, label_pred, average='macro'))
print("precision : ", precision_score(label_true, label_pred, average='macro'))

import matplotlib.pyplot as plt
from matplotlib.colors import *
 
def plot_confusion_matrix(cm, classes,
                        normalize=True,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
target_names =["Legit","Malicious"]
plot_confusion_matrix(cm=matrix, classes=target_names, normalize=False)

from matplotlib import pyplot
 
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()
 
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()

# softsign 64

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from keras.metrics import *
import keras_metrics as km

model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1:]), return_sequences=False, activation='softsign'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile( loss="binary_crossentropy",
              optimizer=RMSprop(lr=0.001), 
              metrics=['accuracy'])

history = model.fit(x=X_train, y=Y_train,batch_size=36,
                    epochs=50, validation_split = 0.2, shuffle=True)

from sklearn.metrics import *
y_pred = model.predict(X_test)

label_pred = np.argmax(y_pred, axis = -1)
label_true = np.argmax(Y_test, axis = -1)
matrix = confusion_matrix(label_true, label_pred)
print(matrix)
print("acc : ", accuracy_score(label_true, label_pred))
print("f1 : ", f1_score(label_true, label_pred, average='macro'))
print("recall : ", recall_score(label_true, label_pred, average='macro'))
print("precision : ", precision_score(label_true, label_pred, average='macro'))

import matplotlib.pyplot as plt
from matplotlib.colors import *
 
def plot_confusion_matrix(cm, classes,
                        normalize=True,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
target_names =["Legit","Malicious"]
plot_confusion_matrix(cm=matrix, classes=target_names, normalize=False)

from matplotlib import pyplot
 
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()
 
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()

# softsign 128

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from keras.metrics import *
import keras_metrics as km

model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1:]), return_sequences=False, activation='softsign'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile( loss="binary_crossentropy",
              optimizer=RMSprop(lr=0.001), 
              metrics=['accuracy'])

history = model.fit(x=X_train, y=Y_train,batch_size=36,
                    epochs=50, validation_split = 0.2, shuffle=True)

from sklearn.metrics import *
y_pred = model.predict(X_test)

label_pred = np.argmax(y_pred, axis = -1)
label_true = np.argmax(Y_test, axis = -1)
matrix = confusion_matrix(label_true, label_pred)
print(matrix)
print("acc : ", accuracy_score(label_true, label_pred))
print("f1 : ", f1_score(label_true, label_pred, average='macro'))
print("recall : ", recall_score(label_true, label_pred, average='macro'))
print("precision : ", precision_score(label_true, label_pred, average='macro'))

import matplotlib.pyplot as plt
from matplotlib.colors import *
 
def plot_confusion_matrix(cm, classes,
                        normalize=True,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
target_names =["Legit","Malicious"]
plot_confusion_matrix(cm=matrix, classes=target_names, normalize=False)

from matplotlib import pyplot
 
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()
 
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()

# softsign 512

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization
from keras.metrics import *
import keras_metrics as km

model = Sequential()
model.add(LSTM(512, input_shape=(X_train.shape[1:]), return_sequences=False, activation='softsign'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile( loss="binary_crossentropy",
              optimizer=RMSprop(lr=0.001), 
              metrics=['accuracy'])

history = model.fit(x=X_train, y=Y_train,batch_size=36,
                    epochs=50, validation_split = 0.2, shuffle=True)

from sklearn.metrics import *
y_pred = model.predict(X_test)

label_pred = np.argmax(y_pred, axis = -1)
label_true = np.argmax(Y_test, axis = -1)
matrix = confusion_matrix(label_true, label_pred)
print(matrix)
print("acc : ", accuracy_score(label_true, label_pred))
print("f1 : ", f1_score(label_true, label_pred, average='macro'))
print("recall : ", recall_score(label_true, label_pred, average='macro'))
print("precision : ", precision_score(label_true, label_pred, average='macro'))

import matplotlib.pyplot as plt
from matplotlib.colors import *
 
def plot_confusion_matrix(cm, classes,
                        normalize=True,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
target_names =["Legit","Malicious"]
plot_confusion_matrix(cm=matrix, classes=target_names, normalize=False)

from matplotlib import pyplot
 
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()
 
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'])
pyplot.show()